A notebook to test quadratic MMD two-sample test

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import freqopttest.util as util
import freqopttest.data as data
import freqopttest.kernel as kernel
import freqopttest.tst as tst
import freqopttest.glo as glo
import sys
import time

In [ ]:
# sample source 
n = 20000
dim = 1500
seed = 14
alpha = 0.01
ss = data.SSGaussMeanDiff(dim, my=1)
#ss = data.SSGaussVarDiff(dim)
#ss = data.SSSameGauss(dim)
#ss = data.SSBlobs()
dim = ss.dim()
tst_data = ss.sample(n, seed=seed)
tr, te = tst_data.split_tr_te(tr_proportion=0.5, seed=10)

## Test permutations

In [ ]:
xtr, ytr = tr.xy()
xytr = tr.stack_xy()
sig2 = util.meddistance(xytr, subsample=1000)
k = kernel.KGauss(sig2)

In [ ]:
start = time.time()

perm_mmds1 = tst.QuadMMDTest.permutation_list_mmd2(xtr, ytr, k, n_permute=20)

end = time.time()
print('permutations took: %.4f s'%(end-start))
print('perm_mmds1', perm_mmds1)

In [ ]:
# test h1_mean_var
start = time.time()
mean, var = tst.QuadMMDTest.h1_mean_var(xtr, ytr, k, is_var_computed=True)
end = time.time()
print('h1_mean_var took: %.3f'%(end - start))
print('mean: %.3g, var: %.3g'%(mean, var))

In [ ]:
# test h1_mean_var_gram
start = time.time()
Kx = k.eval(xtr, xtr)
Ky = k.eval(ytr, ytr)
Kxy = k.eval(xtr, ytr)
mean, var = tst.QuadMMDTest.h1_mean_var_gram(Kx, Ky, Kxy, k, True)
end = time.time()
print('h1_mean_var took: %.3f'%(end - start))
print('mean: %.3g, var: %.3g'%(mean, var))

In [ ]:
# choose the best parameter and perform a test with permutations
med = util.meddistance(tr.stack_xy(), 1000)
list_gwidth = np.hstack( ( (med**2) *(2.0**np.linspace(-4, 4, 40) ) ) )
list_gwidth.sort()
list_kernels = [kernel.KGauss(gw2) for gw2 in list_gwidth]

# grid search to choose the best Gaussian width
besti, powers = tst.QuadMMDTest.grid_search_kernel(tr, list_kernels, alpha)
# perform test 
best_ker = list_kernels[besti]

In [ ]:
start = time.time()
mmd_test = tst.QuadMMDTest(best_ker, n_permute=10, alpha=alpha)
test_result = mmd_test.perform_test(te)
end = time.time()
print 'MMD test took: %s seconds'%(end-start)